In [1]:
from pyapp.model_connection.lm.langchain.litellm import get_model_mlflow_llamacpp, ModelConfig
### --- config --- ###
from pathlib import Path
from dotenv import load_dotenv
import os 
here = Path("./").resolve()
print(Path(here / "config.env").exists())
load_dotenv(here / "config.env")

mlflow_port = os.getenv("MLFLOW_PORT") or "5001"
tracking_uri=f"http://localhost:{mlflow_port}"
config_path=here / "model_dir"

### --- model --- ###
model_config = ModelConfig(model_name="rag_model",port=8080,gguf_relative_path="model_path/artifacts/model.gguf")
model = get_model_mlflow_llamacpp(tracking_uri,config_path,model_config , stream=True , mock_response="Hello world", temperature=0, max_tokens=20)

True
May 7, 2025 > 17:03:09 |  pyapp.serve_integration.mlflow_llamacpp : 23 |   ℹ️ INFO  |  Config path: /Users/parsa/Desk/projects/university/slmops-project/slmops-thesis/app_projects/app/pyapp/notebooks/model_dir  
May 7, 2025 > 17:03:09 |  pyapp.serve_integration.mlflow_llamacpp : 24 |   ℹ️ INFO  |  MLflow client: <mlflow.tracking.client.MlflowClient object at 0x10925efc0>  
May 7, 2025 > 17:03:09 |  pyapp.serve_integration.mlflow_llamacpp : 26 |   ℹ️ INFO  |  Experiments: [<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1745927919661, experiment_id='0', last_update_time=1745927919661, lifecycle_stage='active', name='Default', tags={}>]  
May 7, 2025 > 17:03:09 |  serve.servers.llamacpp.serve : 54 |   ℹ️ INFO  |  Initialized LLaMA CPP server manager  
May 7, 2025 > 17:03:09 |  serve.utils.model_config : 66 |   ℹ️ INFO  |  Initialized model configuration manager with config path: /Users/parsa/Desk/projects/university/slmops-project/slmops-thesis/app_projects/app/p

In [6]:
from opentelemetry.trace import SpanKind
from pyapp.observation.phoneix import PhonexLangChainInstrumentor
observation = PhonexLangChainInstrumentor("lmorbits-phoenix","app2")
tracer = observation.get_tracer("__name__")



Attempting to instrument while already instrumented
Overriding of current TracerProvider is not allowed


In [9]:
from loguru import logger
from taskpy import TaskCLI
from pathlib import Path
tracer = None

class PhoenixObservation:
  @property
  def port(self):
    return 6006

  @property
  def grpc_port(self):
    return 4317

  def __init__(self):
    self.task = TaskCLI(Path(__file__).parent )
  
  
  def is_running(self):
      try:
        answer = self.task.run("status")
        return int(answer.stdout.strip())!= 0
      except Exception as e:
        logger.error(f"Error in phoenix observation: {e}")
        return False 

  def start(self):
    try:
      status = self.is_running()
      if status:
        logger.info("Phoenix is already running")
      else: 
        self.task.run("start", port=self.port, grpc_port=self.grpc_port)
    except Exception as e:
      logger.error(f"Error in phoenix observation: {e}")

  def stop(self):
    try:
      status = self.is_running()
      if not status:
        logger.info("Phoenix is not running")
      else:
        self.task.run("stop")
    except Exception as e:
      logger.error(f"Error in phoenix observation: {e}")
  
  def remove(self):
    try:
      self.task.run("remove")
    except Exception as e:
      logger.error(f"Error in phoenix observation: {e}")


class PhonexLangChainInstrumentor:
  tracer_provider = None

  def __init__(self, project_name: str= "lmorbits-phoenix" , app_name: str= "app1"):
    try:
      self.app_name = app_name
      self.project_name = project_name
      from opentelemetry import trace
      from openinference.instrumentation.langchain import LangChainInstrumentor
      from phoenix.otel import register

      self.tracer_provider = register(
        project_name=self.project_name,
        set_global_tracer_provider=False,
        verbose=False
      ) 
      LangChainInstrumentor().instrument(tracer_provider=self.tracer_provider)
      trace.set_tracer_provider(self.tracer_provider)
      

    except Exception as e:
      logger.error(f"Error in phoenix observation: {e}")
  
  def get_tracer(self , module_name:str):
    from opentelemetry import trace
    return trace.get_tracer(f"{self.app_name}.{module_name}")
    
  
  


Attempting to instrument while already instrumented


In [12]:
from openinference.instrumentation import using_metadata
metadata = {
    "key-1": "value_1",
    "key-2": "value_2",
}

@using_metadata(metadata)
def call_fn(*args, **kwargs):
    print("call_fn")

In [ ]:
from openinference.instrumentation import TraceConfig


https://docs.arize.com/phoenix/tracing/how-to-tracing/setup-tracing/setup-sessions

In [40]:

from openinference.instrumentation import using_attributes, using_user , using_metadata, using_session
from openinference.semconv.trace import SpanAttributes
import uuid
from opentelemetry import trace

tracer = trace.get_tracer("1")


async def app1(message:str, session_id:str):

    with tracer.start_as_current_span(f"app1",kind=SpanKind.INTERNAL) as app1_span:
        app1_span.set_attribute(SpanAttributes.SESSION_ID, session_id)
        app1_span.set_attribute(SpanAttributes.INPUT_VALUE, message)
        with using_session(session_id):
            answer = ""
            async for chunk in model.astream(message):
                answer += chunk.content
                print(chunk, end="|", flush=True)
    
        app1_span.set_attribute(SpanAttributes.OUTPUT_VALUE, answer)
    return answer

# tracer2 = trace.get_tracer("2")
# @tracer2.start_as_current_span(f"app2",kind=SpanKind.INTERNAL, attributes={SpanAttributes.OPENINFERENCE_SPAN_KIND: "agent"})
# async def app2(message:str, session_id:str):
#     current_span = trace.get_current_span()
#     current_span.set_attribute(SpanAttributes.SESSION_ID, session_id)
#     current_span.set_attribute(SpanAttributes.INPUT_VALUE, message)
#     with using_session(session_id):
#         answer = ""
#         async for chunk in model.astream(message):
#             answer += chunk
#             print(chunk, end="|", flush=True)
    
#     current_span.set_attribute(SpanAttributes.OUTPUT_VALUE, answer)
#     return answer

# prompt_template = "Please describe the weather forecast for {city} on {date}"
# prompt_template_variables = {"city": "Johannesburg", "date":"July 11"}
# prompt_template_version = "v1.0"
# with using_attributes(
#     session_id="my-session-id",
#     user_id="my-user-id",
#     metadata=metadata,
#     tags=tags,
#     prompt_template=prompt_template,
#     prompt_template_version=prompt_template_version,
#     prompt_template_variables=prompt_template_variables,
# ):
    

with tracer.start_as_current_span(f"test",kind=SpanKind.CLIENT,) as main_span:
    session_id = str(uuid.uuid4())
    await app1("hello", session_id)
# app2("i", session_id)



content='\n' additional_kwargs={} response_metadata={'model_name': 'openai/custom'} id='run--e538bff6-985f-454a-ba3a-deeace29e737' usage_metadata={'input_tokens': 0, 'output_tokens': 0, 'total_tokens': 0}|content='```' additional_kwargs={} response_metadata={} id='run--e538bff6-985f-454a-ba3a-deeace29e737' usage_metadata={'input_tokens': 0, 'output_tokens': 0, 'total_tokens': 0}|content='\n' additional_kwargs={} response_metadata={} id='run--e538bff6-985f-454a-ba3a-deeace29e737' usage_metadata={'input_tokens': 0, 'output_tokens': 0, 'total_tokens': 0}|content='\n' additional_kwargs={} response_metadata={} id='run--e538bff6-985f-454a-ba3a-deeace29e737' usage_metadata={'input_tokens': 0, 'output_tokens': 0, 'total_tokens': 0}|content='I' additional_kwargs={} response_metadata={} id='run--e538bff6-985f-454a-ba3a-deeace29e737' usage_metadata={'input_tokens': 0, 'output_tokens': 0, 'total_tokens': 0}|content=' have' additional_kwargs={} response_metadata={} id='run--e538bff6-985f-454a-ba3a-

In [ ]:
import uuid

import openai
from openinference.instrumentation import using_session
from openinference.semconv.trace import SpanAttributes
from opentelemetry import trace

session_id = str(uuid.uuid4())

tracer = trace.get_tracer(__name__)

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("user", "{input}"),
])

chain = prompt | model

@tracer.start_as_current_span(name="app1", attributes={SpanAttributes.OPENINFERENCE_SPAN_KIND: "agent"})
def assistant(
  messages: list[dict],
  session_id: str = str,
):
  current_span = trace.get_current_span()
  current_span.set_attribute(SpanAttributes.SESSION_ID, session_id)
  current_span.set_attribute(SpanAttributes.INPUT_VALUE, messages[-1].get('content'))

  # Propagate the session_id down to spans crated by the OpenAI instrumentation
  # This is not strictly necessary, but it helps to correlate the spans to the same session
  with using_session(session_id):
     response = chain.invoke(messages)
     

  current_span.set_attribute(SpanAttributes.OUTPUT_VALUE, response.content)
  return response

@tracer.start_as_current_span(name="app2", attributes={SpanAttributes.OPENINFERENCE_SPAN_KIND: "agent"})
def assistant2(
  messages: list[dict],
  session_id: str = str,
):
  current_span = trace.get_current_span()
  current_span.set_attribute(SpanAttributes.SESSION_ID, session_id)
  current_span.set_attribute(SpanAttributes.INPUT_VALUE, messages[-1].get('content'))

  # Propagate the session_id down to spans crated by the OpenAI instrumentation
  # This is not strictly necessary, but it helps to correlate the spans to the same session
  with using_session(session_id):
     response = chain.invoke(messages)
     

  current_span.set_attribute(SpanAttributes.OUTPUT_VALUE, response.content)
  return response

with tracer.start_as_current_span(f"test",kind=SpanKind.CLIENT,) as main_span:
    messages = [
        {"role": "user", "content": "hi! im bob"}
    ]
    response = assistant(
      messages,
      session_id=session_id,
    )
    response2 = assistant2(
      messages,
      session_id=session_id,
    )
# messages = messages + [
#   response,
#   {"role": "user", "content": "what's my name?"}
# ]
# response = assistant(
#   messages,
#   session_id=session_id,
# )

In [46]:
messages = [
  {"role": "user", "content": "hi! im bob"}
]
answer = chain.invoke(messages)

In [48]:
answer.content

"[{'role': 'user', 'content': 'hi! im bob'}]\n\n```"

In [20]:
with tracer.start_as_current_span(f"test",kind=SpanKind.CLIENT,) as main_span: 
    main_span.add_event("test event")
    with tracer.start_as_current_span(f"app1",metadata=dict(app="app1")) as app1_span: 
        async for chunk in model.astream("just say hello"):
            print(chunk, end="|", flush=True)
    with tracer.start_as_current_span(f"app2",kind=SpanKind.INTERNAL) as app2_span: 
        async for chunk in model.astream("just say hello"):
            print(chunk, end="|", flush=True)

I0000 00:00:1746628828.377568  568758 chttp2_transport.cc:1201] ipv6:%5B::1%5D:4317: Got goaway [11] err=UNAVAILABLE:GOAWAY received; Error code: 11; Debug Text: ping_timeout {created_time:"2025-05-07T17:40:28.377561+03:00", http2_error:11, grpc_status:14}


TypeError: OITracer.start_as_current_span() got an unexpected keyword argument 'metadata'

In [ ]:
SpanKind.INTERNAL

In [4]:
from pyapp.observation.phoneix import PhoenixObservation

phoenix = PhoenixObservation()
answer = phoenix.start()


2025-05-07 16:35:04.252 | DEBUG    | taskpy.main:run:28 - ['task', '--dir', PosixPath('/Users/parsa/Desk/projects/university/slmops-project/slmops-thesis/app_projects/app/pyapp/src/pyapp/observation'), 'status']
2025-05-07 16:35:04.296 | ERROR    | taskpy.main:run:32 - task: [status] docker ps | grep lmorbits-obeservation | wc -l

2025-05-07 16:35:04.296 | INFO     | taskpy.main:run:34 -        0

2025-05-07 16:35:04.297 | DEBUG    | taskpy.main:run:28 - ['task', '--dir', PosixPath('/Users/parsa/Desk/projects/university/slmops-project/slmops-thesis/app_projects/app/pyapp/src/pyapp/observation'), 'start', 'PORT=6006', 'GRPC_PORT=4317']
2025-05-07 16:35:06.206 | ERROR    | taskpy.main:run:32 - task: [start] docker pull arizephoenix/phoenix:latest
task: [start] docker run -d --name lmorbits-obeservation -p 6006:6006 -p 4317:4317 arizephoenix/phoenix:latest

2025-05-07 16:35:06.207 | INFO     | taskpy.main:run:34 - latest: Pulling from arizephoenix/phoenix
Digest: sha256:3d1baca9d83f83e403

In [2]:
phoenix.stop()

2025-05-07 16:01:17.790 | DEBUG    | taskpy.main:run:28 - ['task', '--dir', PosixPath('/Users/parsa/Desk/projects/university/slmops-project/slmops-thesis/app_projects/app/pyapp/src/pyapp/observation'), 'status']
2025-05-07 16:01:17.839 | ERROR    | taskpy.main:run:32 - task: [status] docker ps | grep lmorbits-obeservation | wc -l

2025-05-07 16:01:17.839 | INFO     | taskpy.main:run:34 -        0

2025-05-07 16:01:17.839 | INFO     | pyapp.observation.phoneix:stop:32 - Phoenix is not running


In [2]:
phoenix.remove()

2025-05-07 16:34:55.461 | DEBUG    | taskpy.main:run:28 - ['task', '--dir', PosixPath('/Users/parsa/Desk/projects/university/slmops-project/slmops-thesis/app_projects/app/pyapp/src/pyapp/observation'), 'remove']
2025-05-07 16:34:56.106 | ERROR    | taskpy.main:run:32 - task: [stop] docker stop lmorbits-obeservation
task: [remove] docker rm lmorbits-obeservation

2025-05-07 16:34:56.106 | INFO     | taskpy.main:run:34 - lmorbits-obeservation
lmorbits-obeservation



In [3]:

dir(answer)

['__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'args',
 'check_returncode',
 'returncode',
 'stderr',
 'stdout']

In [4]:
answer.stdout()

TypeError: 'str' object is not callable